In [1]:
from labelbox import Client
import requests
from getpass import getpass
from PIL import Image
import numpy as np
from io import BytesIO
from typing import Dict, Any
import os
import cv2

In [2]:
from keras import layers
from keras.layers import MaxPooling2D
from keras.layers import concatenate
from keras.layers import UpSampling2D
from keras import Input
from keras_segmentation.models.model_utils import get_segmentation_model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, Conv3D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

In [3]:
def visualize_mask(image: np.ndarray,
                   tool: Dict[str, Any],
                   alpha: float = 0.5) -> np.ndarray:
    """
    Overlays a mask onto an image
    
    Args:
        image (np.ndarray): image to overlay mask onto
        tool (Dict[str,any]): Dict response from the export
        alpha: How much to weight the mask when adding to the image
    Returns:
        image with a point drawn on it.
    """
    mask = np.array(
        Image.open(BytesIO(requests.get(
            tool["instanceURI"]).content)))[:, :, :1]
    mask = mask.reshape(1080, 1920)
    
    return image, mask
    
#     cv2.addWeighted
#     weighted = cv2.addWeighted(image, alpha, mask, 1 - alpha, 0)
#     image[np.sum(mask, axis=-1) > 0] = weighted[np.sum(mask, axis=-1) > 0]
#     return image

In [ ]:
API_KEY = ''.join(["eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.",
"eyJ1c2VySWQiOiJja214ZmY4dGp2OTNxMDc1N2s2dzRndzY2Ii",
"wib3JnYW5pemF0aW9uSWQiOiJja214NWg5Znd0ZzUxMDg0ODdp",
"NHZxMDE3IiwiYXBpS2V5SWQiOiJja215eG1qNHR5eThsMDc0N2",
"hrdHp3b2ptIiwiaWF0IjoxNjE3Mjg0ODIxLCJleHAiOjIyNDg0",
"MzY4MjF9.uJ1wDekRxrYtie31RqpoiM_08tWThlShUZrkqjfCDXI"])
print(API_KEY)
client = Client(API_KEY)

project = client.get_project("ckmx5jeqo9u4007902mbiz6k2")

In [ ]:
export_url = project.export_labels()

In [ ]:
exports = requests.get(export_url).json()

In [ ]:
print(exports[0])
# print(exports[0]["Label"]["objects"][0])

In [ ]:
nmr = 40
print(f'Length:{len(exports)}\n')
# Example image
images = []
masks = []
for nmr in range(len(exports)):
    export = next(exports)
    image = np.array(
        Image.open(BytesIO(requests.get(export["Labeled Data"]).content)))
    if export["Label"] != {}:
        for tool in export["Label"]["objects"]:
            # All tools have instanceURI but the car was made with the segmentation tool
            imageArr, maskArr = visualize_mask(image, tool)
            print(nmr, maskArr.shape)
            images.append(imageArr)
            masks.append(maskArr)
#         print(image.shape)
#         image = Image.fromarray(imageArr.astype(np.uint8))
#         w, h = image.size
#         image.resize((w // 4, h // 4))
#         display(image)

#         print(maskArr.shape)
#         mask = Image.fromarray(maskArr)
#         w, h = mask.size
#         mask.resize((w // 4, h // 4))
#         display(mask)

In [4]:
input_height = 320
input_width = 640

In [8]:
img_input = Input(shape=(input_height,input_width , 3 ))

conv1 = Conv2D(32, (1, 1), activation='relu', padding='same')(img_input)
conv1 = Dropout(0.6)(conv1)
conv1 = Conv2D(32, (1, 1), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Dropout(0.6)(conv2)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D((2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Dropout(0.6)(conv3)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

up1 = concatenate([UpSampling2D((2, 2))(conv3), conv2], axis=-1)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
conv4 = Dropout(0.6)(conv4)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

up2 = concatenate([UpSampling2D((2, 2))(conv4), conv1], axis=-1)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
conv5 = Dropout(0.6)(conv5)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)

out = Conv2D( 2, (1, 1) , padding='same')(conv5)

model = get_segmentation_model(img_input ,  out ) # this would build the segmentation model

In [9]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 320, 640, 3) 0                                            
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 320, 640, 32) 128         input_2[0][0]                    
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, 320, 640, 32) 0           conv2d_11[0][0]                  
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 320, 640, 32) 1056        dropout_5[0][0]                  
____________________________________________________________________________________________

In [12]:
model.train(
    train_images = '../../../images_zip/images/',
    train_annotations = '../../../images_zip/masks/', 
    epochs = 30
)

  1%|█▊                                                                                                                                                                                | 3/300 [00:00<00:12, 24.59it/s]

Verifying training dataset


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:12<00:00, 23.54it/s]


Dataset verified! 
Epoch 1/30
512/512 [==============================] - 50s 95ms/step - loss: 0.9615 - accuracy: 0.77070s - loss: 0.9622 - ac
Epoch 2/30
512/512 [==============================] - 49s 95ms/step - loss: 0.8059 - accuracy: 0.7725
Epoch 3/30
512/512 [==============================] - 49s 95ms/step - loss: 0.7133 - accuracy: 0.7790
Epoch 4/30
512/512 [==============================] - 49s 96ms/step - loss: 0.6562 - accuracy: 0.7907
Epoch 5/30
512/512 [==============================] - 49s 96ms/step - loss: 0.6040 - accuracy: 0.8053
Epoch 6/30
512/512 [==============================] - 51s 100ms/step - loss: 0.5618 - accuracy: 0.8215
Epoch 7/30
512/512 [==============================] - 53s 103ms/step - loss: 0.5334 - accuracy: 0.8328
Epoch 8/30
512/512 [==============================] - 53s 103ms/step - loss: 0.5162 - accuracy: 0.8424
Epoch 9/30
512/512 [==============================] - 53s 103ms/step - loss: 0.4905 - accuracy: 0.8540
Epoch 10/30
512/512 [================

In [14]:
out = model.predict_segmentation(
    inp="test1.png",
    out_fname="out1_2.png"
)